## Installing DKube SDK

In [ ]:
! pip3 install git+https://github.com/oneconvergence/dkube.git@2.2-dev --upgrade --no-cache-dir
import os
os._exit(00)

In [ ]:
from dkube.sdk import *

In [ ]:
user = os.getenv('USERNAME')
authToken = os.getenv('DKUBE_USER_ACCESS_TOKEN')
dataset_name = "titanic"
pre_inp_path = "/opt/dkube/input"
ptrain_out_path = "/opt/dkube/output/train"
ptest_out_path = "/opt/dkube/output/test"
project_name = "titanic"
training_fs = "titanic-train-fs"
test_fs = "titanic-test-fs"
model_name = "titanic"
framework = "scikit_v0.2.2"
image = "ocdr/d3-datascience-tf-cpu:fs-v1.14"
preprocess_script = "python preprocessing.py --train_fs titanic-train-fs --test_fs titanic-test-fs"
training_script = "python training.py"
transformer_code = "predict.py"

In [ ]:
api = DkubeApi(token=authToken)

In [ ]:
preprocessing_name= generate(project_name)
preprocessing = DkubePreprocessing(user, name=preprocessing_name, description='triggered from dkube sdk')
preprocessing.update_container(image_url=image)
preprocessing.update_startupscript(preprocess_script)
preprocessing.add_project(project_name)
preprocessing.add_input_dataset(dataset_name, mountpath=pre_inp_path)
preprocessing.add_output_featureset(titanic-train-fs, mountpath=ptrain_out_path)
preprocessing.add_output_featureset(titanic-test-fs, mountpath=ptest_out_path)

In [ ]:
api.create_preprocessing_run(preprocessing)

In [ ]:
training_name = generate(project_name)
training = DkubeTraining(user, name = training_name, description='triggered from dkube sdk')
training.update_container(framework = framework, image_url=image)
training.update_startupscript(training_script)
training.add_project(project_name)
training.add_input_featureset(titanic-train-fs, mountpath="/titanic-train")
training.add_input_featureset(titanic-test-fs, mountpath="/titanic-test")
training.add_output_model(model_name, mountpath="/model")

In [ ]:
api.create_training_run(training)

In [ ]:
for each_datum in api.list_models(user = user)[0]["datums"]:
    if each_datum["datum"]["name"] == model_name:
        head_version = each_datum["datum"]["generated"]["head_version"]
        break

In [ ]:
head_version